In [1]:
#importing libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [2]:
import os
#os.getcwd()
os.chdir("C:\\Users\\akadali\\Desktop\\Deep_NLP\\MLG_Capstone_ChatBot\\ChatBot_BOW_NN")

In [ ]:
"""
try:
    #In case of any new data added to the faq.json file, please delete "data.pickle" file from the 
    #directory before running the code. 
    with open("data.pickle", 'rb') as f:
        words, labels, train_x, train_y = pickle.load(f)
"""

In [3]:
#Creating empty lists of words, classes and documents 
words = []
classes = []
documents = []

faq_file = open("faq.json").read()
faq = json.loads(faq_file)

In [4]:
for q in faq["faq"]:
    for pattern in q['patterns']:
        #tokenizing the documents to words
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        #creating documents with token lists and class
        documents.append((tokens, q['tag']))
        if q['tag'] not in classes:
            classes.append(q['tag'])

In [5]:
#print(documents)
#print(len(words))
ignore_letters = ['!', '#', '%', '^', '&', '*', '?', '/']
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
#Printing number of documents, classes and lemmatized words
print("\n")
print(len(documents), " documents")
print("\n")
print(len(classes), " classes")
print("\n")
print(len(words), " unique lemmatized words")

#Creating words and classes pickle files

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))



104  documents


30  classes


195  unique lemmatized words


In [7]:
#Creating Training Dataset with bag of words
training = []

#Creating an empty list to save the one-hot encoded form (BAG OF WORDS) of output classes as target variable
output_empty = [0]*len(classes)

#Creating an empty list to save the one-hot encoded form (BAG OF WORDS) of output classes as target variable
output_empty = [0]*len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for word in words:
        if word in pattern_words:
            bag.append(1)
        else:
            bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [8]:
#Shuffling the training dataset
random.shuffle(training)

training = np.array(training)
#Creating dependent and independent lists 
train_x = list(training[:,0])
train_y = list(training[:,1])

with open("data.pickle", "wb") as f:
    pickle.dump((words, classes, train_x, train_y), f)
print("Training data is ready")

Training data is ready


In [9]:
#Create model - 3 layers. 
#First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(128, input_shape =(len(train_x[0]), ) , activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(64, activation = 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(len(train_y[0]), activation = 'softmax')
                             ])

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = tf.keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot_model.h5', hist)

Epoch 1/200
21/21 [==============================] - 0s 3ms/step - loss: 3.4001 - accuracy: 0.0288
Epoch 2/200
21/21 [==============================] - 0s 3ms/step - loss: 3.3467 - accuracy: 0.0769
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 3.2663 - accuracy: 0.1058
Epoch 4/200
21/21 [==============================] - 0s 3ms/step - loss: 3.0943 - accuracy: 0.1538
Epoch 5/200
21/21 [==============================] - 0s 3ms/step - loss: 2.9750 - accuracy: 0.1923
Epoch 6/200
21/21 [==============================] - 0s 3ms/step - loss: 2.8610 - accuracy: 0.2212
Epoch 7/200
21/21 [==============================] - 0s 3ms/step - loss: 2.7112 - accuracy: 0.2308
Epoch 8/200
21/21 [==============================] - 0s 3ms/step - loss: 2.5542 - accuracy: 0.3654
Epoch 9/200
21/21 [==============================] - 0s 3ms/step - loss: 2.3026 - accuracy: 0.3077
Epoch 10/200
21/21 [==============================] - 0s 3ms/step - loss: 2.1885 - accuracy: 0.3750
Epoch 11/

21/21 [==============================] - 0s 2ms/step - loss: 0.1616 - accuracy: 0.9519
Epoch 84/200
21/21 [==============================] - 0s 2ms/step - loss: 0.2730 - accuracy: 0.9423
Epoch 85/200
21/21 [==============================] - 0s 2ms/step - loss: 0.3177 - accuracy: 0.8942
Epoch 86/200
21/21 [==============================] - 0s 2ms/step - loss: 0.1640 - accuracy: 0.9519
Epoch 87/200
21/21 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.9423
Epoch 88/200
21/21 [==============================] - 0s 2ms/step - loss: 0.2936 - accuracy: 0.9135
Epoch 89/200
21/21 [==============================] - 0s 2ms/step - loss: 0.2736 - accuracy: 0.9135
Epoch 90/200
21/21 [==============================] - 0s 2ms/step - loss: 0.1755 - accuracy: 0.9519
Epoch 91/200
21/21 [==============================] - 0s 2ms/step - loss: 0.1089 - accuracy: 0.9519
Epoch 92/200
21/21 [==============================] - 0s 2ms/step - loss: 0.2693 - accuracy: 0.9135
Epoch 93/200


21/21 [==============================] - 0s 2ms/step - loss: 0.0813 - accuracy: 0.9808
Epoch 165/200
21/21 [==============================] - 0s 2ms/step - loss: 0.1464 - accuracy: 0.9327
Epoch 166/200
21/21 [==============================] - 0s 2ms/step - loss: 0.1598 - accuracy: 0.9423
Epoch 167/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0827 - accuracy: 0.9712
Epoch 168/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0703 - accuracy: 0.9808
Epoch 169/200
21/21 [==============================] - 0s 1ms/step - loss: 0.1231 - accuracy: 0.9615
Epoch 170/200
21/21 [==============================] - 0s 1ms/step - loss: 0.1277 - accuracy: 0.9615
Epoch 171/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0671 - accuracy: 0.9712
Epoch 172/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0991 - accuracy: 0.9615
Epoch 173/200
21/21 [==============================] - 0s 2ms/step - loss: 0.0527 - accuracy: 0.9808
Epoc

In [10]:
def bag_of_words(s, words):
    bag = [0]*len(words)
    s_words = nltk.word_tokenize(s)
    s_words = [lemmatizer.lemmatize(word.lower()) for word in s_words if word not in ignore_letters]
    for word in s_words:
        for i,w in enumerate(words):
            if w == word:
                bag[i] = 1
    return np.array(bag)

In [15]:
#Create an empty list to collect the user queries for further review
userInputQueue = []
botOutputQueue = []
def chat():
    print("Start talking with the bot (type 'quit' to stop)")
    while True:
        user_input = input("You: ")
        userInputQueue.append(user_input)
        if user_input.lower() == 'quit':
            print("Do you want to save your conversation?? [Y/N]\n")
            input_file_flag = input("Your response: ")
            if input_file_flag.lower() == 'y':
                conversation = open("user_inputs.txt", "w")
                for que in userInputQueue:
                    for ans in botOutputQueue:
                        conversation.write("User:")
                        conversation.write(que)
                        conversation.write("\n")
                        conversation.write("Bot:")
                        conversation.write(ans)
                        conversation.write("\n")
                conversation.close()
                print("Your conversation has been saved. Thank you")
            break
        bow = np.array([bag_of_words(user_input, words)])
        results = model.predict(bow)
        #print("Results\n",results)
        result_index = np.argmax(results) 
        #print("Result Index\n",result_index)
        label = classes[result_index]
        
        for q in faq['faq']:
            if q['tag'] == label:
                response = random.choices(q['responses'])
                botOutputQueue.append(response)
                print("Bot:", response[0])
            #default bot response
            #print("I didn't get that, try again.")
#classes[10]

In [16]:
chat()

Start talking with the bot (type 'quit' to stop)
You: Hi
Bot: Hi there, how can I help?
You: I want to know my referral status
Bot: If you submitted your referral in the system, you should be receiving status emails from ustalentreferralprogram@deloitte.com. Else, please reach out to ustalentreferralprogram@deloitte.com, to know your referral status.
You: how can I know my referral status
Bot: If you submitted your referral in the system, you should be receiving status emails from ustalentreferralprogram@deloitte.com. Else, please reach out to ustalentreferralprogram@deloitte.com, to know your referral status.
You: When will I receive my referral bonus
Bot: Thank you for your referral. The review of the referral eligibility will be initiated at the time of the referral starting. The bonus award will be processed after the payment is reviewed and approved.
You: Can you tell me the status of my referral bonus for Carl Nerlich, a director in state credits and incentives
Bot: Thank you for

* import test data
* create test question embeddings
* create a new column for new answers
* get the answers in that column
* compare both answers
* get the results
* Get the cosine values as well